In [72]:
using Revise

In [2]:
using Flux
using JLD2
using FileIO
using MLDataPattern
using ProgressMeter
using Plots; gr()

Plots.GRBackend()

In [3]:
import FluxExtensions
import LearningMPC
import LCPSim
import LCPSim
import Hoppers
using RigidBodyDynamics
using Gurobi
using MeshCatMechanisms, MeshCat

In [4]:
samples = load("../2018-02-07-hopper-smaller-grid/grid_search.jld2")["samples"];

In [35]:
filter!(samples) do sample
    sample.state[2] <= sample.state[1] && !any(isnan, sample.uJ)
end;
length(samples)

59931

In [36]:
features(sample::LearningMPC.Sample) = (sample.state, sample.mip.objective_bound, sample.mip.objective_value)

features (generic function with 1 method)

In [37]:
data = features.(samples);
train_data, test_data = splitobs(shuffleobs(data), at=0.85);

In [55]:
function setup_model()
    model = Chain(
        Dense(4, 32, elu),
        Dense(32, 32, elu),
        Dense(32, 1),
        )

    loss = (x, lb, ub) -> begin
        y = model(x)
        sum(ifelse.(y .< lb, lb .- y, ifelse.(y .> ub, y .- ub, 0 .* y)))
    end
    model, loss
end

model, loss = setup_model()

(Chain(Dense(4, 32, NNlib.elu), Dense(32, 32, NNlib.elu), Dense(32, 1)), #47)

In [56]:
model(train_data[1][1])

Tracked 1-element Array{Float64,1}:
 0.457366

In [57]:
opt = Flux.ADADelta(params(model))

(::#71) (generic function with 1 method)

In [58]:
losses = Tuple{Float64, Float64}[]
push!(losses, 
    (mean(xy -> Flux.Tracker.data(loss(xy...)), train_data),
     mean(xy -> Flux.Tracker.data(loss(xy...)), test_data))
)
@showprogress for i in 1:100
    Flux.train!(loss, train_data, opt)
    push!(losses, 
        (mean(xy -> Flux.Tracker.data(loss(xy...)), train_data),
         mean(xy -> Flux.Tracker.data(loss(xy...)), test_data))
    )
end

Progress: 100%|█████████████████████████████████████████| Time: 0:37:55


In [49]:
train_data

50941-element SubArray{Tuple{Array{Float64,1},Float64,Float64},1,Array{Tuple{Array{Float64,1},Float64,Float64},1},Tuple{Array{Int64,1}},false}:
 ([0.375, 0.375, 0.875, -0.5], 112.562, 118.541)  
 ([1.375, 1.0, -1.625, -0.625], 243.323, 243.363) 
 ([1.0, 0.875, -1.5, 1.625], 206.094, 206.094)    
 ([0.875, 0.5, 1.5, 1.625], 273.888, 273.888)     
 ([1.125, 0.375, -1.375, 0.375], 237.483, 237.483)
 ([1.0, 1.0, -0.5, -1.125], 216.625, 227.778)     
 ([1.25, 0.5, 2.0, -0.375], 262.871, 278.812)     
 ([0.875, 0.25, 0.5, 0.625], 252.997, 252.997)    
 ([0.75, 0.25, -0.25, 0.625], 198.831, 208.674)   
 ([1.125, 0.75, -1.0, 0.625], 226.886, 226.886)   
 ([1.375, 0.75, 0.125, -0.625], 265.243, 278.673) 
 ([1.375, 0.625, 1.5, 1.25], 265.15, 265.21)      
 ([1.5, 0.625, -0.125, 0.625], 267.794, 282.495)  
 ⋮                                                
 ([1.5, 0.875, 2.0, -2.0], 133.958, 134.053)      
 ([1.5, 0.5, -0.625, 2.0], 257.48, 275.692)       
 ([1.375, 0.25, -0.5, -1.875], 278.382, 

In [59]:
plt = plot(first.(losses))
plot!(plt, last.(losses))
ylims!(plt, (0, ylims(plt)[2]))
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
<polyline clip-path="url(#clip5202)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.8198,11.9279 41.2647,349.692 46.7097,351.949 52.1546,353.453 57.5996,356.591 63.0445,359.534 68.4895,361.413 73.9344,362.415 79.3794,363.12 84.8244,363.035 
 90.2693,364.441 95.7143,365.021 101.159,365.449 106.604,366.321 112.049,366.864 117.494,367.075 122.939,367.56 128.384,367.998 133.829,368.483 139.274,368.845 
 144.719,369.066 150.164,369.267 155.609,369.371 161.054,369.537 166.499,369.837 171.944,369.804 177.389,370.087 182.833,370.188 188.278,369.865 193.723,370.291 
 199.168,370.599 204.613,370.735 210.058,370.637 215.503,370.845 220.948,370.604 226.393,370.237 231.838,370.711 237.283,370.407 242.728,370.185 248.173,370.997 
 253.618,370.773 259.063,370.328 264.508,370.216 269.953,369.523 275.398,370.315 280.843,370.456 286.288,370.112 291.733,369.983 297.177,370.268 302.622,370.197 
 308.067,371.037 313.512,370.722 318.957,371.246 324.402,370.548 329.847,371.44 335.292,370.942 340.737,370.08 346.182,370.993 351.627,370.302 357.072,370.354 
 362.517,370.704 367.962,370.294 373.407,370.487 378.852,370.462 384.297,370.109 389.742,371.191 395.187,370.926 400.632,370.296 406.077,370.872 411.521,370.847 
 416.966,370.623 422.411,370.562 427.856,370.354 433.301,370.842 438.746,371.212 444.191,371.007 449.636,371.004 455.081,370.325 460.526,371.114 465.971,371.389 
 471.416,371.314 476.861,371.196 482.306,370.451 487.751,371.643 493.196,371.015 498.641,371.148 504.086,370.798 509.531,371.229 514.976,370.746 520.42,371.089 
 525.865,370.928 531.31,371.279 536.755,370.259 542.2,370.923 547.645,371.351 553.09,371.125 558.535,371.345 563.98,371.459 569.425,371.448 574.87,370.703 
 580.315,370.778 
 "/>
<polyline clip-path="url(#clip5202)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 35.8198,11.811 41.2647,349.157 46.7097,351.328 52.1546,352.793 57.5996,355.965 63.0445,358.946 68.4895,360.848 73.9344,361.875 79.3794,362.604 84.8244,362.553 
 90.2693,363.988 95.7143,364.616 101.159,365.078 106.604,365.991 112.049,366.564 117.494,366.829 122.939,367.37 128.384,367.821 133.829,368.341 139.274,368.728 
 144.719,368.954 150.164,369.165 155.609,369.284 161.054,369.45 166.499,369.741 171.944,369.728 177.389,370.017 182.833,370.122 188.278,369.795 193.723,370.221 
 199.168,370.528 204.613,370.666 210.058,370.568 215.503,370.78 220.948,370.55 226.393,370.17 231.838,370.67 237.283,370.349 242.728,370.139 248.173,370.954 
 253.618,370.725 259.063,370.278 264.508,370.172 269.953,369.492 275.398,370.261 280.843,370.405 286.288,370.076 291.733,369.929 297.177,370.22 302.622,370.155 
 308.067,370.986 313.512,370.681 318.957,371.183 324.402,370.485 329.847,371.369 335.292,370.889 340.737,370.046 346.182,370.939 351.627,370.254 357.072,370.31 
 362.517,370.656 367.962,370.237 373.407,370.446 378.852,370.414 384.297,370.072 389.742,371.133 395.187,370.871 400.632,370.25 406.077,370.83 411.521,370.787 
 416.966,370.582 422.411,370.521 427.856,370.307 433.301,370.792 438.746,371.142 444.191,370.946 449.636,370.962 455.081,370.275 460.526,371.072 465.971,371.318 
 471.416,371.266 476.861,371.144 482.306,370.43 487.751,371.59 493.196,370.992 498.641,371.113 504.086,370.783 509.531,371.195 514.976,370.749 520.42,371.075 
 525.865,370.928 531.31,371.262 536.755,370.282 542.2,370.925 547.645,371.346 553.09,371.13 558.535,371.336 563.98,371.458 569.425,371.46 574.87,370.726 
 580.315,370.801 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

In [24]:
robot = Hoppers.Hopper()
mvis = MechanismVisualizer(robot.mechanism, URDFVisuals(Hoppers.urdf))
IJuliaCell(mvis)

Listening on 127.0.0.1:7002...
zmq_url=tcp://127.0.0.1:6002
web_url=http://127.0.0.1:7002/static/


MeshCat.IJuliaCell(MeshCat.ViewerWindow(ZMQ.Context(Ptr{Void} @0x0000000009f9abc0, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x000000000a254140, Base.Filesystem._FDWatcher(Ptr{Void} @0x000000000a3f5250, 59, (1, 0), Condition(Any[]), (false, false), 0))]), ZMQ.Socket(Ptr{Void} @0x000000000a254140, Base.Filesystem._FDWatcher(Ptr{Void} @0x000000000a3f5250, 59, (1, 0), Condition(Any[]), (false, false), 0)), "http://127.0.0.1:7002/static/", "tcp://127.0.0.1:6002", MeshCat.ZMQServer.ZMQWebSocketBridge(ip"127.0.0.1", Set{WebSockets.WebSocket}(), MeshCat.SceneTrees.SceneNode(Nullable{Array{UInt8,1}}(), Nullable{Array{UInt8,1}}(), Dict("meshcat"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0xa8, 0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74], Dict("world"=>MeshCat.SceneTrees.SceneNode(#NULL, #NULL, Dict("floor"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], Dict("floor"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6c, 0x6f, 0x6f, 0x72, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], Dict("geometry_1"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String,MeshCat.SceneTrees.SceneNode}()))),"core"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x6c, 0x6f, 0x6f, 0x72, 0x2f, 0x63, 0x6f, 0x72, 0x65], Dict("after_base_z"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x65, 0x72, 0x5f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x7a], Dict("geometry_2"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String,MeshCat.SceneTrees.SceneNode}()))),"foot"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x63, 0x6f, 0x72, 0x65, 0x2f, 0x66, 0x6f, 0x6f, 0x74], Dict("after_foot_extension"=>MeshCat.SceneTrees.SceneNode(#NULL, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x5f, 0x65, 0x78, 0x74, 0x65, 0x6e, 0x73, 0x69, 0x6f, 0x6e], Dict("geometry_3"=>MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String,MeshCat.SceneTrees.SceneNode}()))))))))))))))), 7002, HttpServer.Server(HttpServer.HttpHandler(MeshCat.ZMQServer.handle_file_request, Base.TCPServer(RawFD(43) active), Dict{String,Function}(Pair{String,Function}("error", HttpServer.#1),Pair{String,Function}("listen", HttpServer.#3))), WebSockets.WebSocketHandler(MeshCat.ZMQServer.#5)), ZMQ.Context(Ptr{Void} @0x000000000968e760, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x0000000008c417c0, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000009693220, 50, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x000000000a73b140, Base.Filesystem._FDWatcher(Ptr{Void} @0x000000000a2d85c0, 51, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x000000000a25d010, Base.Filesystem._FDWatcher(Ptr{Void} @0x000000000a6a0200, 52, (1, 0), Condition(Any[Task (runnable) @0x00007f0ef549cfd0]), (true, false), 0))]), ZMQ.Socket(Ptr{Void} @0x000000000a25d010, Base.Filesystem._FDWatcher(Ptr{Void} @0x000000000a6a0200, 52, (1, 0), Condition(Any[Task (runnabl

In [61]:
xstar = Hoppers.nominal_state(robot)
ustar = zeros(num_velocities(xstar))

Q, R = Hoppers.default_costs(robot)
foot = findbody(robot.mechanism, "foot")
Δt = 0.05
Jc = LCPSim.ContactLQR.contact_jacobian(xstar, [Point3D(default_frame(foot), 0., 0., 0.)])
A, B, c = LCPSim.ContactLQR.contact_linearize(xstar, ustar, Jc)

([0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], [0.0 0.0; 0.0 0.0; 0.0 1.0; 0.0 1.0], [0.0, 0.0, -9.81, -9.81])

In [62]:
tangent_net = FluxExtensions.TangentPropagator(model)
net_value_controller = state -> begin
    x = state_vector(state)
    value, jac = tangent_net(x)
    u = vec(-inv(R) * B' * Flux.Tracker.data(jac)')
end

(::#59) (generic function with 1 method)

In [76]:
x_init = MechanismState{Float64}(robot.mechanism)
set_configuration!(x_init, [1.0, 1.0])
set_velocity!(x_init, [0., 0.])
set_configuration!(mvis, configuration(x_init))
sleep(3)
# LearningMPC.randomize!(x_init, x_init, 0.5, 1.0)
results = LCPSim.simulate(x_init, net_value_controller,
    robot.environment,
    Δt,
    100,
    GurobiSolver(Gurobi.Env(), OutputFlag=0));
LearningMPC.playback(mvis, results, Δt)

Academic license - for non-commercial use only


In [71]:
LearningMPC.playback(mvis, results, Δt)

In [65]:
using JLD2

In [66]:
jldopen("model.jld2", "w") do file
    file["model"] = model
end

Chain(Dense(4, 32, NNlib.elu), Dense(32, 32, NNlib.elu), Dense(32, 1))